In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences


path = "../Datasets/CoAID/"

dfTrue = pd.read_csv(path+"NewsRealCOVID-19.csv", usecols=['title'], nrows=500)
dfTrue['label']=1
dfFalse = pd.read_csv(path+"NewsFakeCOVID-19.csv", usecols=['title'], nrows=500)
dfFalse['label']=0
dfTotal = pd.concat([dfTrue, dfFalse])

X = dfTotal['title'].values
y = dfTotal['label'].values

X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(
  X, y, test_size=0.25, random_state=450)




## Evaluate on the second external validation dataset

In [9]:
validationNewsDF = pd.read_csv('../Datasets/New_External_Validation_Dataset.csv')

validationX = validationNewsDF['title'].values
validationX = [one_hot(str(elem),10000) for elem in validationX]
validationX = pad_sequences(validationX, padding='post', maxlen=500)
validationY = validationNewsDF['label'].values

#For SVM, LR, and NB, we used the default settings provided in the scikit-learn 
#For CNN we use the standard implementation with default setting https://github.com/dennybritz/cnn-text-classification-tf
#Could just use an out-of-the-box CNN implementation from a TF tutorial
# The dennybritz one seems good but I can do it later
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB
# Bernoulli was best of above
import math
X_train = [one_hot(str(elem), 10000) for elem in X_train_base]
X_test =  [one_hot(str(elem), 10000) for elem in X_test_base]
X_train = pad_sequences(X_train, padding='post', maxlen=500)
X_test = pad_sequences(X_test, padding='post', maxlen=500)

svmClassifier = svm.SVC()
svmClassifier.fit(X_train, y_train_base)
svmPrediction = svmClassifier.predict(X_test)
svmPrediction = [math.floor(0.5+pred) for pred in svmPrediction]

svmValidationPrediction = svmClassifier.predict(validationX)
svmValidationPrediction = [math.floor(0.5+pred) for pred in svmValidationPrediction]
print("CoAID_only: Results of SVM external validation, on our new dataset:")
print(classification_report(validationY, svmValidationPrediction))



X_train = [one_hot(str(elem), 10000) for elem in X_train_base]
X_test =  [one_hot(str(elem), 10000) for elem in X_test_base]
X_train = pad_sequences(X_train, padding='post', maxlen=500)
X_test = pad_sequences(X_test, padding='post', maxlen=500)

lrClassifier = LogisticRegression(random_state=0, max_iter=500)
lrClassifier.fit(X_train, y_train_base)
lrPrediction = lrClassifier.predict(X_test)
lrPrediction = [math.floor(0.5+pred) for pred in lrPrediction]

lrValidationPrediction = lrClassifier.predict(validationX)
lrValidationPrediction = [math.floor(0.5+pred) for pred in lrValidationPrediction]
print("CoAID_only: Results of LR external validation, on our new dataset:")
print(classification_report(validationY, lrValidationPrediction))

X_train = [one_hot(str(elem), 10000) for elem in X_train_base]
X_test =  [one_hot(str(elem), 10000) for elem in X_test_base]
X_train = pad_sequences(X_train, padding='post', maxlen=500)
X_test = pad_sequences(X_test, padding='post', maxlen=500)

bnbClassifier = BernoulliNB()
bnbClassifier.fit(X_train, y_train_base)
bnbPrediction = bnbClassifier.predict(X_test)
bnbPrediction = [math.floor(0.5+pred) for pred in bnbPrediction]

bnbValidationPrediction = bnbClassifier.predict(validationX)
bnbValidationPrediction = [math.floor(0.5+pred) for pred in bnbValidationPrediction]
print("CoAID_only: Results of Bernoulli NB external validation, on our new dataset:")
print(classification_report(validationY, bnbValidationPrediction))



CoAID_only: Results of SVM external validation, on our new dataset:
              precision    recall  f1-score   support

           0       0.99      0.54      0.70     14398
           1       0.71      0.99      0.83     16232

    accuracy                           0.78     30630
   macro avg       0.85      0.77      0.76     30630
weighted avg       0.84      0.78      0.77     30630

CoAID_only: Results of LR external validation, on our new dataset:
              precision    recall  f1-score   support

           0       0.98      0.55      0.70     14398
           1       0.71      0.99      0.83     16232

    accuracy                           0.78     30630
   macro avg       0.85      0.77      0.77     30630
weighted avg       0.84      0.78      0.77     30630

CoAID_only: Results of Bernoulli NB external validation, on our new dataset:
              precision    recall  f1-score   support

           0       0.99      0.56      0.72     14398
           1       0.72  